# Introdução à Inteligência Artificial
## Avaliação Contínua 4

## Lógica Proposicional e CSP

### Entrega: 28 de Novembro às 23:59

## Introdução
O objectivo desta avaliação contínua é modelizar como um Problema de Satisfação de Restrições (CSP) o problema de encontrar um modelo que torna verdadeiro um conjunto de fórmulas proposicionais. O conjunto de fórmulas representa implicitamente uma conjunção dessas fórmulas. Por modelo entende-se o conjunto de afectações de um valor de verdade a cada símbolo proposicional envolvido nas fórmulas de modo a que satisfaçam as fórmulas: cada uma das fórmulas é verdadeira para esse modelo.

Vamos impor a **restrição que essas fórmulas sejam obrigadas a que quando reduzidas à forma normal conjuntiva (FNC) não tenham disjunções que envolvam mais do que dois símbolos proposicionais**. Só assim garantiremos que poderemos formular este problema como um CSP limitado a restrições binárias. Isso não quer dizer que não possamos lidar com fórmulas que envolvam mais do que 2 símbolos proposicionais. Vejam esta fórmula, por exemplo, que tem 3 símbolos proposicionais: P &rarr; (Q &and; R), mas que quando convertida na FNC dará origem a uma conjunção de duas disjunções: (&not;P &or; Q) &and; (&not;P &or; R), em que cada uma delas não envolve mais do que 2 símbolos proposicionais.

### Variáveis
Notem que as variáveis do CSP serão os diversos símbolos proposicionais (no formato de strings) que aparecem na fórmula para a qual queremos encontrar um modelo. Para a fórmula P &rarr; (Q &and; R) teríamos como variáveis P, Q e R. Sabem que na classe CSP as variáveis devem estar numa lista e, para esta formulação, **devem respeitar a ordem lexicográfica**. Neste caso teríamos a lista
```python
['P','Q','R']
``` 

### Domínios
Os domínios das variáveis serão formados pelos dois valores booleanos `True` e `False`. Também os domínios **devem respeitar a ordem lexicográfica**.

### Restrições Unárias
Como sabem, no aima-python a classe CSP não lida com restrições unárias, assume que as restrições unárias já foram lidadas através da filtragem dos domínios em conformidade. No caso das fórmulas proposicionais, serão os literais (fórmulas atómicas negadas ou não) que fazem o papel de restrições unárias. Para lidar com a consistência dos domínios, 
* uma variável que tenha um literal positivo (não negado) obrigará a retirar do seu domínio o valor `False`, e 
* uma variável a que corresponda um literal negado no conjunto de fórmulas, verá removido o valor `True` do respectivo domínio.

### Vizinhança e Restrições Binárias
Como dissemos, teremos de converter o conjunto de fórmulas original na FNC, ficando apenas com um conjunto de disjunções (consideramos o caso de termos apenas um literal como sendo uma disjunção). As fórmulas originais estão obrigadas a gerarem disjunções na FNC que envolvam no máximo 2 variáveis e assim, cada disjunção com 2 símbolos proposicionais é uma restrição entre as respectivas variáveis, criando também a relação de vizinhança.

Assim, no caso de termos ficado com a FNC (&not;P &or; Q) &and; (&not;P &or; R) formada por duas disjunções, teremos que 'P' e 'Q' são vizinhos e o mesmo acontece com 'P' e 'R'.

Notem que **podemos ter mais do que uma disjunção que envolve as mesmas duas variáveis**. É o caso de termos estas duas disjunções: (&not;P &or; &not;Q) &and; (P &or; Q). Nesse caso a função que lida com a satisfação de restrições tem de garantir que os dois valores das variáveis satisfazem todas as disjunções que envolvem os respectivos símbolos proposicionais.

## Objectivo
O objectivo deste projecto é desenvolver a função `csp_prop` em Python que recebe uma fórmula proposicional, de acordo com o ficheiro `logic.py` e do guião de lógica proposicional e que devolva uma instância da classe CSP à qual possamos aplicar os algoritmos de procura em CSP de modo a encontrarmos uma solução (ou modelo que torne a fórmula verdadeira) ou nenhuma solução. 
```python
def csp_prop(formulas):
    pass
```
Para construir uma instância da classe CSP é preciso definir:

* a lista de variáveis; 
* a lista dos valores, i.e., os domínios, das variáveis, com as restrições unárias já filtradas;
* o grafo de vizinhanças;
* a função que verifica se quaisquer duas variáveis e respectivos valores satisfazem ou não todas as suas restrições.

**Uma solução será um modelo, i.e., a afectação de valores às variáveis de modo a que satisfaçam todas as restrições.**

## Exemplos

### Exemplo 1
```python
formulas={expr('A ==> (B & C)'),expr('A')}
abc_csp=csp_prop(formulas)
print(abc_csp.variables)
```
Output:
```python
['A', 'B', 'C']
```

### Exemplo 2
```python
formulas={expr('A ==> (B & C)'),expr('A')}
abc_csp=csp_prop(formulas)
print(abc_csp.domains)
```
O output neste caso deveria ser o dicionário
```python
{'A': [True], 'B': [False, True], 'C': [False, True]}
```
No entanto, para os testes automáticos, este dicionário é convertido para uma lista ordenada para ser possível comparar automaticamente. Não se preocupem que os testes automáticos estão preparados para fazer essa conversão fazendo
```python
print(sorted([(var,sorted(val)) for (var,val) in abc_csp.domains.items()]))
```
e obtendo o resultado
```python
[('A', [True]), ('B', [False, True]), ('C', [False, True])]
```

### Exemplo 3
```python
formulas={expr('A ==> (B & C)'),expr('A')}
abc_csp=csp_prop(formulas)
print(sorted([(var,sorted(val)) for (var,val) in abc_csp.neighbors.items()]))
```
De forma semelhante aos domínos, o output é o dicionário
```python
{'A': ['B', 'C'], 'B': ['A'], 'C': ['A']}
```
convertido para
```python
[('A', ['B', 'C']), ('B', ['A']), ('C', ['A'])]
```

### Exemplo 4
```python
formulas={expr('A & ~A')}
a_csp=csp_prop(formulas)
print(sorted([(var,sorted(val)) for (var,val) in a_csp.neighbors.items()]))
```
Output
```python
[('A', [])]
```

### Exemplo 5
```python
formulas={expr('A ==> (B & C)'),expr('A')}
abc_csp=csp_prop(formulas)
r = backtracking_search(abc_csp)
print('Assignment = ',r) 
```
Output
```python
Assignment =  {'A': True, 'B': True, 'C': True}
```

### Exemplo 6
```python
formulas={expr('VA | VP'),expr('AV | AP'),expr('PV | PA'),expr('VP'),expr('VA ==> ~VP'),expr('AP ==> ~AV'),\
          expr('PA ==> ~PV'),expr('VA ==> ~PA'),expr('PV ==> ~AV'),expr('VP ==> ~AP')}
dancam_csp=csp_prop(formulas)
r = backtracking_search(dancam_csp)
print('Assignment = ',r) 
```
Output
```python
Assignment =  {'AP': False, 'AV': True, 'PA': True, 'PV': False, 'VA': False, 'VP': True}
```

## Submissão

### Quizz

Cada grupo deve completar a implementação da função pedida e testá-la no link do *quizz* **Avaliação Contínua 4** que está na página da disciplina, introduzindo aí o vosso código. Os vários elementos do grupo podem desenvolver e avaliar o código porque no final o que conta é aquele com maior  valor.

Esse *quizz* é constituído por uma única pergunta. A implementação da função `csp_prop` é avaliada através de um conjunto de testes automáticos visíveis e mais alguns testes escondidos, valendo um total de 1.75 valores.

Podem ir verificando o código e submeterem as vezes que quiserem, sendo a submissão com melhor nota a que será considerada.

Os testes visiveis valemr 6 em 20, enquanto que os testes escondidos valem 14 em 20.

### Ficheiro Python

Simultaneamente é necessário submeter o script Python que contém todo o código submetido no *quizz*. Só queremos uma submissão por grupo. Esse ficheiro deve chamar-se **CSP_Prop_IIA_23_24_grupoXX.py** em que substituem XX pelo número do grupo.